In [2]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 数据预处理
train = pd.read_csv("../data/titanic/train.csv")
test = pd.read_csv("../data/titanic/test.csv")
# 删除Name、Ticket、Cabin
train = train.drop(['Name','Ticket','Cabin'], axis=1)

test = test.drop(['Name','Ticket','Cabin'], axis=1)
combine = [train,test]
# 性别数值化
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mean())
    dataset['P1'] = np.array(dataset['Pclass'] == 1).astype(np.int32) 
    dataset['P2'] = np.array(dataset['Pclass'] == 2).astype(np.int32) 
    dataset['P3'] = np.array(dataset['Pclass'] == 3).astype(np.int32) 
    dataset['E1'] = np.array(dataset['Embarked'] == "S").astype(np.int32) 
    dataset['E2'] = np.array(dataset['Embarked'] == "C").astype(np.int32) 
    dataset['E3'] = np.array(dataset['Embarked'] == "Q").astype(np.int32) 
    
for dataset in combine:
    del dataset['Pclass']
    del dataset['Embarked']
    
data_train = combine[0][['Sex','Age','SibSp','Parch','Fare','P1','P2','P3','E1','E2','E3']]
data_test = combine[1][['Sex','Age','SibSp','Parch','Fare','P1','P2','P3','E1','E2','E3']]

data_target = combine[0][['Survived']]

# 模型构建
import tensorflow as tf
x = tf.placeholder('float',shape=[None,11])
y = tf.placeholder('float',shape=[None,1])
weiget = tf.Variable(tf.random_normal([11,1]))
bias = tf.Variable(tf.random_normal([1]))
output = tf.matmul(x,weiget) + bias
pred = tf.cast(tf.sigmoid(output) > 0.5,tf.float32)
# 模型构建
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(lables = y,logits = output))
# 使用梯度下降
train_step = tf.train.GradientDescentOptimizer(0.0003).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, y), tf.float32))

# 运行
sess = tf.Session()
sess.run(tf.global_variables_initializer())
loss_train = []
train_acc = []
test_acc = []

for i in range(25000):
    index = np.random.permutation(len(data_target))
    data_train = data_train[index]
    data_target = data_target[index]
    for n in range(len(data_target)//100+1):
        batch_xs = data_train[n * 100: n * 100 + 100]
        batch_ys = data_target[n * 100: n * 100 + 100]
        sess.run(train_step,feed_dict={x: batch_xs, y: batch_ys})
    if i % 1000 == 0:
        loss_tmp = sess.run(loss,feed_dict={x: batch_xs, y: batch_ys})
        loss_train.append(loss_tmp)
        train_acc_tmp = sess.run(accuracy,feed_dict={x: batch_xs, y: batch_ys})
        train_acc.append(train_acc_tmp)
        print(loss_tmp,train_acc)


     Sex       Age  SibSp  Parch      Fare  P1  P2  P3  E1  E2  E3
0      0  34.50000      0      0    7.8292   0   0   1   0   0   1
1      1  47.00000      1      0    7.0000   0   0   1   1   0   0
2      0  62.00000      0      0    9.6875   0   1   0   0   0   1
3      0  27.00000      0      0    8.6625   0   0   1   1   0   0
4      1  22.00000      1      1   12.2875   0   0   1   1   0   0
5      0  14.00000      0      0    9.2250   0   0   1   1   0   0
6      1  30.00000      0      0    7.6292   0   0   1   0   0   1
7      0  26.00000      1      1   29.0000   0   1   0   1   0   0
8      1  18.00000      0      0    7.2292   0   0   1   0   1   0
9      0  21.00000      2      0   24.1500   0   0   1   1   0   0
10     0  30.27259      0      0    7.8958   0   0   1   1   0   0
11     0  46.00000      0      0   26.0000   1   0   0   1   0   0
12     1  23.00000      1      0   82.2667   1   0   0   1   0   0
13     0  63.00000      1      0   26.0000   0   1   0   1   0

NameError: name 'shape' is not defined